In [8]:
import numpy as np
import os
import re
import random
import json
import sys
import time
from datetime import datetime
import pandas as pd
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')
from tensorflow.keras.models import Model, load_model, clone_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Multiply, Add, Lambda, Concatenate, Reshape, Flatten
from tensorflow.keras.initializers import GlorotUniform, RandomUniform, Constant
from tensorflow.keras.callbacks import LambdaCallback
import jax
import jax.numpy as jnp
from jax import grad, jit

In [9]:
epsilon = 9.5
folder = 'D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/Multidigit_Addition_Decimal/'

In [10]:
# Cargar los módulos preentrenados (unit_module y carry_module)
unit_addition_model = load_model('unit_addition_module.keras')
unit_carry_model = load_model('unit_carry_module.keras')
dec_addition_model = load_model('dec_addition_module.keras')
dec_carry_model = load_model('dec_carry_module.keras')

unit_addition_model.trainable = False
unit_carry_model.trainable = False
dec_addition_model.trainable = False
dec_carry_model.trainable = False

unit_addition_model.name = 'unit_addition_model'
unit_carry_model.name = 'unit_carry_model'
dec_addition_model.name = 'dec_addition_model'
dec_carry_model.name = 'dec_carry_model'

In [11]:
# Cargar las parejas desde el archivo
with open(f"{folder}/test_dataset.txt", 'r') as file:
    test_dataset = eval(file.read())  # Convertir el contenido del archivo a una lista de tuplas
    
with open(f"{folder}/train_couples_stimuli.txt", "r") as file:
    train_couples = eval(file.read())

with open(f"{folder}/combinations_with_carry_over.txt", "r") as file:
    combinations_with_carry_over = eval(file.read())  # Leer y convertir el contenido en una lista de tuplas

with open(f"{folder}/combinations_small_problem_size.txt", 'r') as file:
    combinations_small_problem_size = eval(file.read())

with open(f"{folder}/combinations_large_problem_size.txt", 'r') as file:
    combinations_large_problem_size = eval(file.read())

carry_over_small = [pair for pair in combinations_with_carry_over if pair in combinations_small_problem_size]
carry_over_large = [pair for pair in combinations_with_carry_over if pair in combinations_large_problem_size]
train_with_carry_over = [pair for pair in train_couples if pair in combinations_with_carry_over]
train_with_small = [pair for pair in train_couples if pair in combinations_small_problem_size]
train_with_large = [pair for pair in train_couples if pair in combinations_large_problem_size]
train_with_carry_over_small = [pair for pair in train_couples if pair in combinations_with_carry_over if pair in combinations_small_problem_size]
train_with_carry_over_large = [pair for pair in train_couples if pair in combinations_with_carry_over if pair in combinations_large_problem_size]

In [12]:
def generate_test_dataset():
    x_data = []
    y_data = []
    
    for a, b in test_dataset:
        a_dec = a // 10  # Decena del primer número
        a_unit = a % 10  # Unidad del primer número
        b_dec = b // 10  # Decena del segundo número
        b_unit = b % 10  # Unidad del segundo número

        x_data.append([a_dec, a_unit, b_dec, b_unit])  # Entrada

        sum_units = (a_unit + b_unit) % 10
        carry_units = 1 if (a_unit + b_unit) >= 10 else 0
        sum_dec = (a_dec + b_dec + carry_units) % 10
        carry_dec = 1 if (a_dec + b_dec + carry_units) >= 10 else 0
        y_data.append([carry_dec, sum_dec, sum_units])  # Salida
    
    return jnp.array(x_data), jnp.array(y_data)

# Modelo dinámico en JAX
def model(params, x):
    units_input = jnp.array(x[:, [1, 3]])  # Columnas 1 y 3 representando unidades y decenas
    units_input = units_input[:, None, :]  # Añade una dimensión extra para la secuencia (N, 1, 2)
                            
    unit_output = jnp.array(unit_addition_model(units_input))  # Asegúrate de que la entrada sea un batch
    unit_carry_output = jnp.array(unit_carry_model(units_input))  # Salida de acarreo de unidades

    # Tomar el valor máximo de las predicciones (argmax en JAX)
    unit_val = jnp.argmax(unit_output, axis=-1)
    carry_unit_val = jnp.argmax(unit_carry_output, axis=-1)

    decs_input = jnp.array(x[:, [0, 2]])
    decs_input = jnp.concatenate([decs_input, carry_unit_val[:, None]], axis=-1)
    decs_input = decs_input[:, None, :]  # Añadir dimensión para la secuencia (N, 1, 3)
    
    dec_output = jnp.array(dec_addition_model(decs_input))  # Salida para decenas
    dec_carry_output = jnp.array(dec_carry_model(decs_input))  # Salida de acarreo de decenas
    
    dec_val = jnp.argmax(dec_output, axis=-1)
    carry_dec_val = jnp.argmax(dec_carry_output, axis=-1)

    # Calcular las salidas combinadas con los parámetros v
    salida_1 = (params['v0'] * carry_dec_val) + (params['v1'] * dec_val) + (params['v2'] * carry_unit_val) + (params['v3'] * unit_val)
    salida_2 = (params['v4'] * carry_dec_val) + (params['v5'] * dec_val) + (params['v6'] * carry_unit_val) + (params['v7'] * unit_val)
    salida_3 = (params['v8'] * carry_dec_val) + (params['v9'] * dec_val) + (params['v10'] * carry_unit_val) + (params['v11'] * unit_val)

    return salida_1, salida_2, salida_3
    
    
def predictions(params, x_test, y_test):
    pred_count = 0
    pred_count_train = 0
    pred_count_carry_over = 0
    pred_count_carry_over_train = 0
    pred_count_small = 0
    pred_count_small_train = 0
    pred_count_carry_over_small = 0
    pred_count_carry_over_small_train = 0
    pred_count_large = 0
    pred_count_large_train = 0
    pred_count_carry_over_large = 0
    pred_count_carry_over_large_train = 0
    
    total_examples = x_test.shape[0]
    total_train = len(train_couples)
    total_carry_over = len(combinations_with_carry_over)
    total_train_with_carry_over = len(train_with_carry_over)
    total_small = len(combinations_small_problem_size)
    total_train_with_small = len(train_with_small)
    total_carry_over_small = len(carry_over_small)
    total_train_with_carry_over_small = len(train_with_carry_over_small)
    total_large = len(combinations_large_problem_size)
    total_train_with_large = len(train_with_large)
    total_carry_over_large = len(carry_over_large)
    total_train_with_carry_over_large = len(train_with_carry_over_large)
    
    pred_hundreds, pred_tens, pred_units = model(params, x_test)        
    
    for i in range(total_examples):
        normalized_pred = [int(jnp.round(pred_hundreds[i].item())),
                           int(jnp.round(pred_tens[i].item())),
                           int(jnp.round(pred_units[i].item()))]
        
        a = int(str(x_test[i, 0]) + str(x_test[i, 1]))
        b = int(str(x_test[i, 2]) + str(x_test[i, 3]))
        
        if normalized_pred[0] == y_test[i, 0] and normalized_pred[1] == y_test[i, 1] and normalized_pred[2] == y_test[i, 2]:
            pred_count += 1
            
            if (a, b) in train_couples:
                pred_count_train += 1
                
                if (a, b) in combinations_with_carry_over:
                    pred_count_carry_over_train += 1

                    if (a, b) in combinations_small_problem_size:
                        pred_count_carry_over_small_train += 1

                    elif (a, b) in combinations_large_problem_size:
                        pred_count_carry_over_large_train += 1

                if (a, b) in combinations_small_problem_size:
                    pred_count_small_train += 1

                elif (a, b) in combinations_large_problem_size:
                    pred_count_large_train += 1

            if (a, b) in combinations_with_carry_over:
                pred_count_carry_over +=1

                if (a, b) in combinations_small_problem_size:
                    pred_count_carry_over_small +=1

                elif (a, b) in combinations_large_problem_size:
                    pred_count_carry_over_large +=1
                
            if (a, b) in combinations_small_problem_size:
                pred_count_small +=1   

            elif (a, b) in combinations_large_problem_size:
                pred_count_large +=1 

    print(f'Out of {total_examples}, {pred_count} predictions correct.')
    print(f'Out of {total_train}, {pred_count_train} train predictions correct.')
    print(f'Out of {total_carry_over}, {pred_count_carry_over} carry-over predictions correct.')
    print(f'Out of {total_train_with_carry_over}, {pred_count_carry_over_train} carry-over train predictions correct.')
    print(f'Out of {total_small}, {pred_count_small} small predictions correct.')
    print(f'Out of {total_train_with_small}, {pred_count_small_train} small train predictions correct.')
    print(f'Out of {total_carry_over_small}, {pred_count_carry_over_small} carry-over small predictions correct.')
    print(f'Out of {total_train_with_carry_over_small}, {pred_count_carry_over_small_train} carry-over small train predictions correct.')   
    print(f'Out of {total_large}, {pred_count_large} large predictions correct.')
    print(f'Out of {total_train_with_large}, {pred_count_large_train} large train predictions correct.')
    print(f'Out of {total_carry_over_large}, {pred_count_carry_over_large} carry-over large predictions correct.')
    print(f'Out of {total_train_with_carry_over_large}, {pred_count_carry_over_large_train} carry-over large train predictions correct.') 

    return None

In [13]:
def load_params_from_file(filename):
    with open(filename, 'r') as f:
        return json.load(f)

class Tee(object):
    def __init__(self, file, mode='w'):
        self.file = open(file, mode)
        self.console = sys.stdout  

    def write(self, data):
        self.console.write(data)   
        self.file.write(data)    

    def flush(self):
        self.console.flush()
        self.file.flush()

    def close(self):
        self.file.close()

In [14]:
x_test, y_test = generate_test_dataset()

save_dir = f"{folder}Super_Tests/"
folder_path = f'{folder}Super_Trained_models/AP_{epsilon}'
date_pattern = r'super_trained_model_(\d{4}_\d{2}_\d{2}_\d{2}_\d{2}_\d{2}).json'
files = sorted(
    (f for f in os.listdir(folder_path) if not f.startswith('.')),  # Filtrar archivos ocultos
    key=lambda x: re.search(date_pattern, x).group(1) if re.search(date_pattern, x) else ''
)

os.makedirs(save_dir, exist_ok=True) 
results_file = os.path.join(save_dir, f"Super_Tests_AP_{epsilon}.txt") 
tee = Tee(results_file, 'w') 
sys.stdout = tee

try:
    for filename in files:
        match = re.search(date_pattern, filename)
        if match:
            current_time = match.group(1)
        else:
            print('Error')
            break
        
        file_path = f"{folder_path}/super_trained_model_{current_time}.json"
        with open(file_path, 'rb') as file:
            trained_model = json.load(file)
    
        trained_model_jnp = {key: jnp.array(value) for key, value in trained_model.items()}
        print(f'Loaded super_trained_model_{current_time}.json')

        predictions(trained_model_jnp, x_test, y_test)
    
finally:
    sys.stdout = tee.console
    tee.close()

Loaded super_trained_model_2024_12_20_01_52_02.json
Out of 10000, 352 predictions correct.
Out of 9808, 340 train predictions correct.
Out of 6975, 26 carry-over predictions correct.
Out of 6879, 26 carry-over train predictions correct.
Out of 820, 46 small predictions correct.
Out of 724, 46 small train predictions correct.
Out of 270, 0 carry-over small predictions correct.
Out of 222, 0 carry-over small train predictions correct.
Out of 8109, 234 large predictions correct.
Out of 8013, 222 large train predictions correct.
Out of 6246, 26 carry-over large predictions correct.
Out of 6198, 26 carry-over large train predictions correct.
Loaded super_trained_model_2024_12_20_01_52_04.json
Out of 10000, 337 predictions correct.
Out of 9808, 327 train predictions correct.
Out of 6975, 95 carry-over predictions correct.
Out of 6879, 95 carry-over train predictions correct.
Out of 820, 34 small predictions correct.
Out of 724, 34 small train predictions correct.
Out of 270, 0 carry-over sma